In [1]:
import sys
sys.path.append('../scripts')
from join_data import *
import collections
import cPickle as pickle
import extract_metadata
import numpy as np
import os
import random
import scipy.sparse
import sklearn.datasets
import sklearn.feature_extraction
import time
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

In [64]:
INPUT_DATA_DIR = '/Users/205341/Documents/git/machine-learning/appeals/data'
OUTPUT_DATA_DIR = '/Users/205341/Documents/git/machine-learning/appeals/data'
RESULT_PATH = '../results/model_results.pkl.' + datetime.now().strftime('%Y%m%d-%H%M%S')

NUM_OPINION_SHARDS = 500 #1340
MIN_REQUIRED_COUNT = 50 #50
USE_TFIDF = True
CODED_FEATURE_NAMES = None


# Model params
TRAIN_PCT = 0.75
REG_MIN_LOG10 = -2
REG_MAX_LOG10 = 2
SCORING = 'accuracy'
# NOTE: this will be too slow to run locally if feature reduction is enabled
FEATURE_REDUCTION_TYPE = None # TODO try 'chi2' or l1svc

DESCRIPTION = '.'.join([datetime.now().strftime('%Y%m%d-%H%M%S'), 'min_required_count', str(MIN_REQUIRED_COUNT), 
                        FEATURE_REDUCTION_TYPE if FEATURE_REDUCTION_TYPE else 'all_features', SCORING]) 
RESULT_PATH = RESULT_PATH + '.' + DESCRIPTION

print 'Experiment:', DESCRIPTION

Experiment: 20150510-005042.min_required_count.50.all_features.accuracy


In [3]:
(input_data_dir,
 output_data_dir,
 num_opinion_shards,
 min_required_count, tfidf) = (
    INPUT_DATA_DIR,OUTPUT_DATA_DIR,
    NUM_OPINION_SHARDS,
    MIN_REQUIRED_COUNT,USE_TFIDF)

In [98]:
CASE_DATA_FILENAME = 'merged_caselevel_data.csv'
cases_df = extract_metadata.extract_metadata(input_data_dir + '/' + CASE_DATA_FILENAME)

In [111]:
cases_df.loc[cases_df.duplicated('caseid'),:]

,caseid,direct1,liberal,conservative,mixed,unknown,geniss,casetyp1,treat,majvotes,...,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2
2652,XABD65,1,0,1,0,0,7,759,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4007,X3NUQ5,1,0,1,0,0,7,754,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12255,X3H4N9,3,1,0,0,0,7,705,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14577,X4OUEL,1,0,1,0,0,1,122,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16666,X3M8PD,1,0,1,0,0,2,209,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16667,X3F1RB,1,0,1,0,0,2,209,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16797,X3IRNH,3,1,0,0,0,1,110,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
cases_df.loc[cases_df['caseid']=='XABD65',:]

,caseid,direct1,liberal,conservative,mixed,unknown,geniss,casetyp1,treat,majvotes,...,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2
2651,XABD65,1,0,1,0,0,7,773,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2652,XABD65,1,0,1,0,0,7,759,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df2 = cases_df
df2.loc[df2['direct1'].isin([1,3]),:]

,caseid,direct1,liberal,conservative,mixed,unknown,geniss,casetyp1,treat,majvotes,...,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2
0,X43COQ,3,1,0,0,0,2,202,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,X4AI87,3,1,0,0,0,7,720,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,XAFK3N,1,0,1,0,0,7,731,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,XAFJS6,3,1,0,0,0,7,760,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,X46B2T,1,0,1,0,0,2,291,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,XAC0CM,3,1,0,0,0,2,204,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,X46CBN,3,1,0,0,0,2,291,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,X4AHF9,1,0,1,0,0,1,113,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,X46BAC,3,1,0,0,0,2,241,10,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,X3I6FM,3,1,0,0,0,7,750,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
5 in [1,5]

True

In [82]:
X, case_ids, y,filtered_cases_df,PARAMETERS_DICT = load_data(
                              INPUT_DATA_DIR, OUTPUT_DATA_DIR,
                              NUM_OPINION_SHARDS, MIN_REQUIRED_COUNT,
                              USE_TFIDF, CODED_FEATURE_NAMES
                            )

Data parameters:
  Number of opinion shards: 500
  Minimum required count: 50
  Using TF-IDF: True
Constructing data from scratch...
Reading input data from from: /Users/205341/Documents/git/machine-learning/appeals/data
Parsing opinion shard files...
Computing total n-gram counts...
Writing n-gram counts to disk...
Filtering n-grams...
coded_feature_names:  None
Coded Feature Array shape:  (0,)
Building sparse matrix...
Running tfidf transformation...
shape:  (6607, 20401)
number of cases 6607
Writing input data to disk...
Feature matrix saved as /Users/205341/Documents/git/machine-learning/appeals/data/feature_matrix.svmlight.shards.500.mincount.50.tfidf
Case IDs saved as /Users/205341/Documents/git/machine-learning/appeals/data/case_ids.shards.p.500.mincount.50.tfidf
Total time spent building data: 495.096252918


In [83]:
X_full,y_full,case_ids_full=X,y,case_ids
COLUMN = 'geniss'
MIN_SAMPLES = 40 #must be greater than the number of k-folds for CV
column_vals = sorted(filtered_cases_df[COLUMN].unique())

In [97]:
print case_ids[6183:6187]
print f_case_ids[6183:6187]

['X36B9O', 'X3M8PD', 'X36CV1', 'X36D9Q']
['X36B9O', 'X3M8PD', 'X3M8PD', 'X36CV1']


In [96]:
f_case_ids = filtered_cases_df.index.tolist()
for i in range(len(f_case_ids)):
    if case_ids[i]!=f_case_ids[i]:
        print i,case_ids[i],f_case_ids[i]

6185 X36CV1 X3M8PD
6186 X36D9Q X36CV1
6187 X36AVR X36D9Q
6188 X4QHQN X36AVR
6189 X36BU9 X4QHQN
6190 X36CAD X36BU9
6191 X4QHS5 X36CAD
6192 X36CG0 X4QHS5
6193 X36CUK X36CG0
6194 X4QHNO X36CUK
6195 X36CMV X4QHNO
6196 X3M8SB X36CMV
6197 X4QHSA X3M8SB
6198 X36D0L X4QHSA
6199 X36D04 X36D0L
6200 X36CVC X36D04
6201 X36BU6 X36CVC
6202 X3OD4N X36BU6
6203 X3M946 X3OD4N
6204 X3OD6G X3M946
6205 X3AUVL X3OD6G
6206 X3LT81 X3AUVL
6207 X3OCQ9 X3LT81
6208 X3AV0F X3OCQ9
6209 X3F1V9 X3AV0F
6210 X3F3NN X3F1V9
6211 X3LTFN X3F3NN
6212 X3AV04 X3LTFN
6213 X3F1RG X3AV04
6214 X3F2AO X3F1RG
6215 X3F1T3 X3F2AO
6216 X3LSPJ X3F1T3
6217 X3F1T5 X3LSPJ
6218 X3F2S1 X3F1T5
6219 X3F351 X3F2S1
6220 X3F39I X3F351
6221 X3F376 X3F39I
6222 X3F4A4 X3F376
6223 X3F3MB X3F4A4
6224 X3F3A6 X3F3MB
6225 X3F3OF X3F3A6
6226 X3F3Q2 X3F3OF
6227 X3F4LH X3F3Q2
6228 X3F428 X3F4LH
6229 X3F42T X3F428
6230 X3F3QL X3F42T
6231 X3F4D4 X3F3QL
6232 X3F68V X3F4D4
6233 X3F6RB X3F68V
6234 X3F758 X3F6RB
6235 X3F6GD X3F758
6236 X3F6UH X3F6GD
6237 X3INDR 

IndexError: list index out of range

In [71]:
def stratify_data(X_full,y_full,filtered_cases_df,column,value):
    idx = np.where(filtered_cases_df[column]==value)[0]
    case_ids = filtered_cases_df.iloc[idx].index.tolist()
    print "idx",idx
    print "X_full.shape",X_full.shape
    X = X_full[idx,:]
    y = y_full[idx]
    assert X.shape[0]==len(y), "stratify_data: X and y not same shape"
    return X,y,case_ids

val=6.0
X,y,case_ids = stratify_data(X_full, y_full,filtered_cases_df,COLUMN,val)

idx [ 204  337  368  644  687  768  786  788  790  810  852  855  892  905  931
  947  964  997 1003 1029 1033 1062 1066 1070 1073 1091 1119 1125 1133 1141
 1143 1154 1170 1171 1174 1186 1187 1199 1222 1231 1237 1256 1264 1284 1295
 1301 1320 1350 1356 1375 1380 1401 1405 1407 1412 1423 1432 1458 1480 1495
 1501 1505 1516 1519 1543 1544 1548 1553 1571 1583 1598 1604 1611 1618 1623
 1665 1676 1677 1680 1689 1691 1695 1718 1719 1727 1750 1817 1819 1821 1828
 1840 1846 1865 1870 1871 1873 1874 1875 1880 1935 1947 1949 1954 1970 1976
 1983 1986 1988 1992 2021 2023 2026 2037 2039 2047 2057 2063 2066 2067 2073
 2075 2089 2097 2107 2125 2126 2137 2142 2144 2158 2162 2164 2170 2196 2198
 2205 2206 2211 2216 2229 2232 2233 2241 2252 2255 2277 2279 2282 2285 2289
 2293 2296 2300 2318 2325 2327 2330 2339 2348 2362 2378 2386 2387 2398 2407
 2412 2414 2416 2448 2481 2486 2498 2534 2539 2543 2600 2616 2625 2638 2650
 2651 2660 2668 2686 2692 2702 2706 2708 2710 2715 2726 2732 2736 2749 2750
 2752 27

In [115]:
strat_column='geniss'
strat_column_vals = sorted(filtered_cases_df[strat_column].unique().tolist())

In [185]:
def drop_mixed_labels(X_train,y_train,case_ids_train):
    ix = np.in1d(y_train, [1,3])
    #idx = list(np.where(ix)[0])
    case_ids = list(np.array(case_ids_train)[ix])
    X = X_train[ix,:]
    y = y_full[ix]
    assert X.shape[0]==len(y), "drop_mixed_labels failed: X and y not same shape"
    assert X.shape[0]==len(case_ids), "drop_mixed_labels failed: X and cases_df not same shape"
    return X,y,case_ids

X2,y2,case_ids2=drop_mixed_labels(X,y,case_ids)

In [173]:
X2[:10].todense()

matrix([[ 0.01046233,  0.04333127,  0.0208246 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.0329357 , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

In [182]:
ix = np.in1d(y, [1,3])
ix[:10]

array([ True,  True, False, False, False,  True,  True,  True,  True,  True], dtype=bool)

In [183]:
case_ids[:10]

['X53H9R',
 'X9VBVI',
 'X9VAUC',
 'X9VC1M',
 'X9VB6O',
 'X9VC1C',
 'X9VBOR',
 'X53GMC',
 'X3T509',
 'X53GSR']

In [184]:
case_ids2[:10]

['X53H9R',
 'X9VBVI',
 'X9VC1C',
 'X9VBOR',
 'X53GMC',
 'X3T509',
 'X53GSR',
 'X53GDC',
 'X9VAMS',
 'X53G1P']

In [70]:


print "X.shape",X.shape
print "\n %s = %s" %(COLUMN,val)
if X.shape[0]>=MIN_SAMPLES:
    # TODO write a loop for all of the params
    print 'Training and scoring models...'

    train_and_score_model(X, y, case_ids, 'svm', train_pct=TRAIN_PCT,
                          reg_min_log10=REG_MIN_LOG10, reg_max_log10=REG_MAX_LOG10, scoring=SCORING, feature_reduction_type=FEATURE_REDUCTION_TYPE,
                          result_path=RESULT_PATH, description=DESCRIPTION,
                          parameters_dict = PARAMETERS_DICT)

X.shape (509, 20401)

 geniss = 6.0
Training and scoring models...


NameError: name 'train_and_score_model' is not defined

In [52]:
X2.shape

(7, 1354)

In [57]:
np.where(filtered_cases_df

,direct1,liberal,conservative,mixed,unknown,geniss,casetyp1,treat,majvotes,dissent,...,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2
caseid,,,,,,,,,,,,,,,,,,,,,
X3CF37,0,0,0,0,1,7,764,5,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X9VC3B,2,0,0,1,0,7,740,6,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3FTTR,1,0,1,0,0,2,222,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3H2S1,3,1,0,0,0,7,703,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3SD7C,3,1,0,0,0,7,751,3,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3IR69,1,0,1,0,0,7,740,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3JA7R,3,1,0,0,0,7,727,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X4AAHO,1,0,1,0,0,7,742,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3JI5K,1,0,1,0,0,9,912,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
idx = np.where(filtered_cases_df['geniss']==7)[0]
len(idx)

64

In [63]:
filtered_cases_df.iloc[idx]

,direct1,liberal,conservative,mixed,unknown,geniss,casetyp1,treat,majvotes,dissent,...,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2
caseid,,,,,,,,,,,,,,,,,,,,,
X3CF37,0,0,0,0,1,7,764,5,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X9VC3B,2,0,0,1,0,7,740,6,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3H2S1,3,1,0,0,0,7,703,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3SD7C,3,1,0,0,0,7,751,3,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3IR69,1,0,1,0,0,7,740,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3JA7R,3,1,0,0,0,7,727,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X4AAHO,1,0,1,0,0,7,742,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X4086B,3,1,0,0,0,7,725,2,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3BFCI,1,0,1,0,0,7,733,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
filtered_cases_df['geniss'].unique()

array([ 7.,  2.,  9.,  1.,  6.,  0.,  3.,  4.])

In [35]:
'''
LOAD DATA
'''
start_time = time.time()

print 'Data parameters:'
print '  Number of opinion shards:', num_opinion_shards
print '  Minimum required count:', min_required_count
print '  Using TF-IDF:', tfidf

# Look to see if the data files have already been saved to disk.
feature_matrix_filename, case_ids_filename = build_filenames_from_params(
    output_data_dir, num_opinion_shards, min_required_count, tfidf)

if os.path.isfile(feature_matrix_filename) and os.path.isfile(case_ids_filename):
    with open(feature_matrix_filename, 'rb') as f:
        sparse_feature_matrix, valences = sklearn.datasets.load_svmlight_file(f)
    with open(case_ids_filename, 'rb') as f:
        case_ids = pickle.load(f)
    print 'Data loaded from', feature_matrix_filename, 'and', case_ids_filename
else:
    print 'Constructing data from scratch...'
    print 'Reading input data from from:', input_data_dir
    cases_df = extract_metadata.extract_metadata(input_data_dir + '/' + CASE_DATA_FILENAME)
    opinion_data_dir = input_data_dir + '/' + OPINION_DATA_DIR
    # <HACK> to save the total counts dict to a file
    global GLOBAL_INPUT_DATA_DIR
    GLOBAL_INPUT_DATA_DIR = input_data_dir
    # </HACK>

Data parameters:
  Number of opinion shards: 100
  Minimum required count: 20
  Using TF-IDF: True
Data loaded from /Users/205341/Documents/git/machine-learning/appeals/data/feature_matrix.svmlight.shards.100.mincount.20.tfidf and /Users/205341/Documents/git/machine-learning/appeals/data/case_ids.shards.p.100.mincount.20.tfidf


In [40]:
cases_df = extract_metadata.extract_metadata(input_data_dir + '/' + CASE_DATA_FILENAME)
opinion_data_dir = input_data_dir + '/' + OPINION_DATA_DIR

In [41]:
'''
CONSTRUCT CSR MATRIX
'''
case_ids_df = cases_df['caseid']

rows_ngram_counts, case_ids = parse_opinion_shards(
    case_ids_df, opinion_data_dir, num_opinion_shards)

if min_required_count > 1:
    rows_ngram_counts, case_ids = filter_infrequent_ngrams(
        rows_ngram_counts, case_ids, min_required_count)

rows_ngram_counts, case_ids = sort_case_lists(cases_df, rows_ngram_counts, 
                                              case_ids)

Parsing opinion shard files...
Computing total n-gram counts...
Filtering n-grams...


In [80]:
'''GET CODED DATA'''
coded_features = ['geniss','casetyp1']
#coded_features = 'geniss'

#cast as list
if isinstance(coded_features, str):
    coded_features = [coded_features]

UNKNOWN_VALENCE = 0
NEUTRAL_VALENCE = 2
valences = []
coded_feature_list = []
for case_id in case_ids[:10]:
    valence = int(cases_df[cases_df['caseid'] == case_id]['direct1'].values[0])
    
    if coded_features is not None:
        coded_feature_row = cases_df[cases_df['caseid'] == case_id][coded_features].values[0]
        coded_feature_list.append(coded_feature_row)
    # Replacing unknown valence variables with netural scores.
    if valence == UNKNOWN_VALENCE:
        valence = NEUTRAL_VALENCE
    valences.append(valence)
    
print np.array(coded_feature_list)

[[   7.  734.]
 [   7.  710.]
 [   0.    0.]
 [   7.  731.]
 [   7.  742.]
 [   7.  734.]
 [   7.  740.]
 [   1.  111.]
 [   7.  741.]
 [   7.  727.]]


In [206]:
def get_coded_data(cases_df, case_ids, coded_feature_names):
    """
    Retreives the valences corresponding to case_ids, 
    along with coded features, if any
    Recode unknown valences to neutral.
    args:
      cases_df: A dataframe containing the case variables.
      case_ids: list of sorted case_ids
      coded_feature_names: list of column names to pull from cases_df (ie 'geniss' or ['geniss','casetyp1'])
    returns:
      valences: np array of valences
      coded_feature_array: np array of coded features
    """
    UNKNOWN_VALENCE = 0
    NEUTRAL_VALENCE = 2
    
    if isinstance(coded_feature_names, str):
        coded_feature_names = [coded_feature_names]

    valences = []
    coded_feature_list = []
    for case_id in case_ids:
        valence = int(cases_df[cases_df['caseid'] == case_id]['direct1'].values[0])

        if coded_feature_names is not None:
            coded_feature_row = cases_df[cases_df['caseid'] == case_id][coded_feature_names].values[0]
            clean_row = []

            #clean row
            for val in coded_feature_row:
                if val:
                    clean_row.append(int(val))
                else:
                    clean_row.append(0)
            assert clean_row[0]>=0, ""
            coded_feature_list.append(clean_row)
            
        # Replacing unknown valence variables with netural scores.
        if valence == UNKNOWN_VALENCE:
            valence = NEUTRAL_VALENCE
        valences.append(valence)

    #one-hot encoding
    if coded_feature_names is not None:
        enc = OneHotEncoder()
        coded_feature_array=enc.fit_transform(np.array(coded_feature_list))
    else: coded_feature_array = []
    return np.array(valences),coded_feature_array

In [207]:
valences,features = get_coded_data(cases_df,case_ids,'geniss')

In [202]:
valences

array([2, 2, 1, 3, 3, 1, 3, 1, 1, 3, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 1,
       3, 2, 3, 1, 2, 2, 1, 1, 3, 3, 1, 1, 1, 3, 1, 2, 3, 2, 3, 3, 1, 3, 1,
       1, 3, 3, 2, 1, 3, 1, 1, 1, 1, 3, 3, 2, 3, 2, 3, 1, 3, 1, 3, 3, 2, 1,
       2, 3, 2, 3, 3, 1, 3, 3, 1, 2, 1, 3, 1, 1, 1, 3, 3, 1, 1, 2, 2, 1, 3,
       1, 1, 1, 1, 3, 2, 3, 3, 1, 1, 1, 3, 1, 2, 3, 3, 1, 1, 2, 3, 1, 3, 1,
       1, 3, 1, 1, 1, 1, 3, 2, 1, 1])

In [179]:
def construct_sparse_opinion_matrix(cases_df, opinion_data_dir,
                                    num_opinion_shards,
                                    min_required_count,
                                    tfidf,
                                    coded_feature_names):
    """
    Builds a CSR sparse matrix containing the n-gram counts from the court
    opinion shard files. Also returns the corresponding case_ids and valences.
    The rows of these lists are sorted in order of the dates of the
    corresponding cases, oldest to newest.
    NOTE: This takes ~10 minutes to run with all shards on my macbook.

    Args:
      cases_df: A dataframe containing the case variables. Must include caseid, 
        year, month, day, and direct1.
      opinion_data_dir: The directory where the opinion n-gram shard files
        reside.
      num_opinion_shards: The number of opinion shard files to read in. 
      min_required_count: The minimum number of of times an n-gram must appear
        throughout all documents in order to be included in the data.
      tfidf: Boolean. If set, the returned feature matrix has been normalized
        using TF-IDF.
      coded_feature_names: None or list of features to include from the coded data set.

    Returns:
      sparse_feature_matrix: A scipy.sparse.csr_matrix with n-gram counts.
      ordered_case_ids: A list of case ID strings. The index of each case ID
        corresponds to the index of corresponding row of n-grams in
        sparse_feature_matrix.
      valences: A list of valences as ints, with the unknown valences (0) 
        replaced with the neutral valence (2).
    """
    case_ids_df = cases_df['caseid']

    rows_ngram_counts, case_ids = parse_opinion_shards(
        case_ids_df, opinion_data_dir, num_opinion_shards)

    if min_required_count > 1:
        rows_ngram_counts, case_ids = filter_infrequent_ngrams(
            rows_ngram_counts, case_ids, min_required_count)

    rows_ngram_counts, case_ids = sort_case_lists(cases_df, rows_ngram_counts, 
                                                  case_ids)

    valences,coded_feature_matrix = get_coded_data(cases_df, case_ids,coded_feature_names)

    print 'Building sparse matrix...'
    # Make sure the matrics created by this vectorizer are sparse.
    # set sort=False so that the ordering of the rows by date is preserved.
    dict_vectorizer = sklearn.feature_extraction.DictVectorizer(
        sparse=True, sort=False)
    sparse_feature_matrix = dict_vectorizer.fit_transform(rows_ngram_counts)

    if tfidf:
        print 'Running tfidf transformation...'
        transformer = sklearn.feature_extraction.text.TfidfTransformer()
        sparse_feature_matrix = transformer.fit_transform(sparse_feature_matrix)

    if coded_feature_names is not None:
        print 'Including coded features:', coded_feature_names
        assert sparse_feature_matrix.shape[0]==coded_feature_matrix.shape[0], "Matrices have different number of rows.  N-gram matrix has %r, coded_feature_matrix has %r" %(sparse_feature_matrix.shape[0],coded_feature_matrix.shape[0])
        sparse_feature_matrix = hstack([sparse_feature_matrix,coded_feature_matrix],format='csr')

    assert sparse_feature_matrix.get_shape()[0] == len(case_ids)
    assert len(valences) == len(case_ids)

    print 'shape: ', sparse_feature_matrix.get_shape()
    print 'number of cases', len(case_ids)

    return sparse_feature_matrix, case_ids, valences

In [203]:
coded_feature_names='geniss'
sparse_feature_matrix, case_ids, valences = construct_sparse_opinion_matrix(
            cases_df, opinion_data_dir, 
            10, min_required_count, 
            tfidf,coded_feature_names)

Parsing opinion shard files...
Computing total n-gram counts...
Filtering n-grams...
[[7], [7], [2], [7], [7], [7], [7], [7], [9], [7], [7], [7], [7], [7], [7], [7], [7], [7], [7], [7], [1], [7], [7], [6], [7], [6], [1], [1], [0], [7], [7], [7], [7], [1], [1], [1], [9], [7], [6], [7], [7], [7], [7], [7], [7], [1], [1], [7], [7], [0], [9], [7], [1], [1], [7], [2], [7], [7], [1], [1], [7], [7], [1], [6], [1], [2], [7], [7], [3], [7], [7], [1], [6], [7], [1], [2], [7], [2], [6], [7], [7], [1], [1], [7], [7], [3], [1], [2], [2], [7], [1], [7], [1], [7], [1], [1], [7], [7], [7], [7], [7], [1], [1], [7], [2], [7], [2], [3], [1], [7], [7], [2], [2], [9], [6], [9], [7], [1], [1], [2], [4], [1], [4], [1], [7]]
Building sparse matrix...
Running tfidf transformation...
Including coded features: geniss
shape:  (125, 1362)
number of cases 125


In [204]:
sparse_feature_matrix, case_ids, valences = construct_sparse_opinion_matrix(
            cases_df, opinion_data_dir, 
            10, min_required_count, 
            tfidf,coded_feature_names=None)

Parsing opinion shard files...
Computing total n-gram counts...
Filtering n-grams...
Building sparse matrix...
Running tfidf transformation...
shape:  (125, 1354)
number of cases 125


In [153]:
enc.feature_indices_

array([ 0, 10])

In [160]:
enc.active_features_

array([0, 1, 2, 3, 4, 5, 6, 7, 9])

In [110]:
features[:10]

array([[ 7.],
       [ 7.],
       [ 0.],
       [ 7.],
       [ 7.],
       [ 7.],
       [ 7.],
       [ 1.],
       [ 7.],
       [ 7.]])

In [107]:
print [cases_df.loc[cases_df['caseid']==case_id,['caseid','direct1','geniss']] for case_id in case_ids[:10]]

[       caseid  direct1  geniss
13045  X53G1P        1       7,        caseid  direct1  geniss
13049  X53H3H        1       7,        caseid  direct1  geniss
13074  X53G1S        0       0,        caseid  direct1  geniss
13100  X3T4TU        3       7,        caseid  direct1  geniss
13030  X3IKD4        1       7,        caseid  direct1  geniss
13029  X3IV6M        3       7,        caseid  direct1  geniss
13142  X3T501        0       7,        caseid  direct1  geniss
13033  X3U0CR        3       1,        caseid  direct1  geniss
13037  X3N6P7        1       7,        caseid  direct1  geniss
13048  X3N6HB        3       7]


In [130]:
from scipy.sparse import csr_matrix, hstack
A = csr_matrix([[1, 2], [3, 4]])
B = np.array([[5], [6]])
C = hstack([A,B],format='csr')

In [136]:
B.shape[0]

2

In [197]:
foo = [0,4.5,5,'',4]

In [198]:
bar = []
for x in foo:
    if x:
        bar.append(int(x))
    else: bar.append(0)
print bar

[0, 4, 5, 0, 4]
